In [4]:

import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from scipy.optimize import minimize

# 1. Historical Data for Function 3 (Index 2)
# Extracted from the provided files for Weeks 1-3
X = np.array([
    [0.1, 0.1, 0.1],                 # Week 1
    [0.2, 0.2, 0.2],                 # Week 2
    [0.123105, 0.025252, 0.134332],
    [0.94888554, 0.96563203, 0.80839735]
])

y = np.array([
    -0.09338622073872602,  # Week 1 Output
    -0.16230936451166192,  # Week 2 Output
    -0.1662102640375241,    # Week 3 Output
    -0.07863721776440244
])

# 2. Train Gaussian Process
# Matern kernel allows for non-linear relationships. 
# WhiteKernel handles potential noise in the observations.
kernel = Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=1e-5)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
gp.fit(X, y)

# 3. Define Acquisition Function (Upper Confidence Bound)
# We balance maximizing the mean prediction with exploring high uncertainty (std).
# kappa=2.576 corresponds to roughly 99% confidence exploration.
def acq_func(x, gp, kappa=2.576):
    x = x.reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # We return negative because 'minimize' finds the lowest value
    return -(mean + kappa * std)

# 4. Optimize Next Submission
# We search for inputs between 0 and 1 (positive numbers constraint).
bounds = [(1e-6, 1.0)] * 3
best_x = None
best_val = np.inf

# Run optimization from multiple random starting points to avoid local optima
np.random.seed(42)
for _ in range(50):
    x0 = np.random.uniform(0, 1, 3)
    res = minimize(acq_func, x0, args=(gp,), bounds=bounds, method='L-BFGS-B')
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

print("--- Week 4 Prediction ---")
print(f"Recommended Submission for Function 3: {best_x}")
print(f"Predicted Value (Acquisition Score): {-best_val}")

--- Week 4 Prediction ---
Recommended Submission for Function 3: [1. 1. 1.]
Predicted Value (Acquisition Score): 0.02522384969352018


In [6]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from scipy.optimize import minimize

# 1. Historical Data for Function 3 (Index 2)
X = np.array([
    [0.1, 0.1, 0.1],                 # Week 1
    [0.2, 0.2, 0.2],                 # Week 2
    [0.123105, 0.025252, 0.134332],  # Week 3
    [0.94888554, 0.96563203, 0.80839735]  # Week 4
])

y = np.array([
    -0.09338622073872602,  # Week 1 Output
    -0.16230936451166192,  # Week 2 Output
    -0.1662102640375241,   # Week 3 Output
    -0.07863721776440244   # Week 4 Output
])

# 2. Train Gaussian Process
# Use ConstantKernel to allow the GP to scale to any output magnitude
# Matern kernel with adaptive length_scale_bounds for larger input domains
kernel = ConstantKernel(1.0, constant_value_bounds=(1e-3, 1e3)) * \
         Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2), nu=2.5) + \
         WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1))

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
gp.fit(X, y)

# 3. Define Acquisition Function (Upper Confidence Bound)
def acq_func(x, gp, kappa=2.576):
    x = x.reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    return -(mean + kappa * std)

# 4. Optimize Next Submission
# Set upper bound much higher - can be adjusted based on domain knowledge
# Using 10.0 as a reasonable starting point, but can go higher
UPPER_BOUND = 100.0  # Adjust this as needed (can be 100, 1000, etc.)

bounds = [(1e-6, UPPER_BOUND)] * 3

best_x = None
best_val = np.inf

# Run optimization from multiple random starting points
np.random.seed(42)
for _ in range(100):  # Increased iterations for larger search space
    # Sample from a broader distribution
    x0 = np.random.uniform(0, UPPER_BOUND, 3)
    res = minimize(acq_func, x0, args=(gp,), bounds=bounds, method='L-BFGS-B')
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

# Also try some points near existing best observations
for x_observed in X:
    # Explore around observed points with some scaling
    for scale in [0.5, 1.0, 1.5, 2.0, 3.0]:
        x0 = x_observed * scale
        x0 = np.clip(x0, 1e-6, UPPER_BOUND)
        res = minimize(acq_func, x0, args=(gp,), bounds=bounds, method='L-BFGS-B')
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

print("--- Week 5 Prediction ---")
print(f"Recommended Submission for Function 3: {best_x}")
print(f"Predicted Mean: {gp.predict(best_x.reshape(1,-1))[0]:.6f}")
mean, std = gp.predict(best_x.reshape(1,-1), return_std=True)
print(f"Predicted Std: {std[0]:.6f}")
print(f"Acquisition Score (UCB): {-best_val:.6f}")

--- Week 5 Prediction ---
Recommended Submission for Function 3: [26.0557643  40.92214319 23.80908706]
Predicted Mean: 0.000000
Predicted Std: 0.123949
Acquisition Score (UCB): 0.319293
